In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e10/sample_submission.csv
/kaggle/input/playground-series-s5e10/train.csv
/kaggle/input/playground-series-s5e10/test.csv
/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_10k.csv
/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_2k.csv
/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_100k.csv


In [2]:
!uv pip install -U autogluon > /dev/null

Using Python 3.11.13 environment at: /usr
Resolved 222 packages in 6.31s
⠙ Preparing packages... (0/144)
⠙ Preparing packages... (0/144)
⠙ Preparing packages... (0/144)
botocore   ------------------------------     0 B/13.48 MiB
⠙ Preparing packages... (0/144)
botocore   ------------------------------ 14.88 KiB/13.48 MiB
⠙ Preparing packages... (0/144)
pandas     ------------------------------     0 B/12.20 MiB
botocore   ------------------------------ 14.88 KiB/13.48 MiB
⠙ Preparing packages... (0/144)
plotly     ------------------------------     0 B/9.38 MiB
pandas     ------------------------------     0 B/12.20 MiB
botocore   ------------------------------ 14.88 KiB/13.48 MiB
⠙ Preparing packages... (0/144)
scikit-learn ------------------------------     0 B/9.27 MiB
plotly     ------------------------------     0 B/9.38 MiB
pandas     ------------------------------     0 B/12.20 MiB
botocore   ------------------------------ 14.88 KiB/13.48 MiB
⠙ Preparing packages... (0/144)
matp

In [3]:
from autogluon.tabular import TabularDataset, TabularPredictor
from pathlib import Path

In [4]:
path = Path('/kaggle/input/playground-series-s5e10/')
path

PosixPath('/kaggle/input/playground-series-s5e10')

In [5]:
train_df = pd.read_csv(path/'train.csv',index_col='id')
test_df = pd.read_csv(path/'test.csv',index_col='id')
sub_df = pd.read_csv(path/'sample_submission.csv')
og_df_a = pd.read_csv('/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_100k.csv')

In [6]:
og_df_a

,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,rural,2,0.29,70,night,rainy,False,True,evening,False,False,1,0.64
1,highway,1,0.34,25,dim,clear,False,False,morning,False,False,3,0.27
2,rural,2,0.76,70,night,foggy,True,False,evening,True,True,1,0.76
3,rural,3,0.37,70,night,foggy,True,False,morning,False,True,0,0.60
4,highway,3,0.39,45,dim,rainy,False,True,morning,False,False,0,0.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,rural,2,0.61,60,dim,foggy,False,True,evening,False,False,1,0.54
99996,rural,2,0.52,35,dim,foggy,True,True,afternoon,False,False,1,0.18
99997,rural,2,0.08,70,daylight,clear,False,False,evening,True,False,1,0.20
99998,rural,1,0.83,70,dim,foggy,False,True,morning,False,True,0,0.58


In [7]:
train_df

,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
id,,,,,,,,,,,,,
0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13
1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35
2,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30
3,highway,4,0.07,35,dim,rainy,True,True,morning,False,False,1,0.21
4,rural,1,0.58,60,daylight,foggy,False,False,evening,True,False,1,0.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...
517749,highway,4,0.10,70,daylight,foggy,True,True,afternoon,False,False,2,0.32
517750,rural,4,0.47,35,daylight,rainy,True,True,morning,False,False,1,0.26
517751,urban,4,0.62,25,daylight,foggy,False,False,afternoon,False,True,0,0.19


In [8]:
TabularPredictor.fit??

In [9]:
target = 'accident_risk'
eval_metric = 'rmse'
#train_data = train
train_data = train_df
#Time_limit = 3600*11
Time_limit = 900
problem_type='regression'

In [10]:
predictor = TabularPredictor(
    label=target, 
    eval_metric=eval_metric,
    verbosity=1,
    problem_type=problem_type
).fit(
    train_data, 
    presets='best_quality',
    excluded_model_types=['KNN'], 
    time_limit=Time_limit,
    dynamic_stacking=False,  # ← ADD THIS
    ag_args_fit={'num_gpus': 1}
)

No path specified. Models will be saved in: "AutogluonModels/ag-20251028_141038"
		module 'pyarrow.lib' has no attribute 'ArrayStatistics'
Detailed Traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/autogluon/tabular/trainer/abstract_trainer.py", line 2171, in _train_and_save
    model = self._train_single(**model_fit_kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/autogluon/tabular/trainer/abstract_trainer.py", line 2055, in _train_single
    model = model.fit(X=X, y=y, X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test, total_resources=total_resources, **model_fit_kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/autogluon/core/models/abstract/abstract_model.py", line 1046, in fit
    kwargs = self._preprocess_fit_args(**kwargs)
      

In [11]:
%%time
predictor.leaderboard()

CPU times: user 7.69 ms, sys: 10 µs, total: 7.7 ms
Wall time: 8.68 ms


,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-0.056236,root_mean_squared_error,55.013692,716.369876,0.009311,0.369644,3,True,6
1,WeightedEnsemble_L2,-0.056368,root_mean_squared_error,45.125673,417.726172,0.008993,0.287876,2,True,4
2,ExtraTrees_r42_BAG_L1,-0.056415,root_mean_squared_error,15.592199,116.130691,15.592199,116.130691,1,True,3
3,ExtraTreesMSE_BAG_L1,-0.056417,root_mean_squared_error,14.883768,141.503731,14.883768,141.503731,1,True,2
4,RandomForestMSE_BAG_L2,-0.056443,root_mean_squared_error,55.004381,716.000232,9.887701,298.561936,2,True,5
5,RandomForestMSE_BAG_L1,-0.056598,root_mean_squared_error,14.640713,159.803874,14.640713,159.803874,1,True,1


In [12]:
%%time
autogluon_preds = predictor.predict(test_df)

CPU times: user 46.6 s, sys: 3.69 s, total: 50.3 s
Wall time: 16.2 s


In [13]:
autogluon_preds

id
517754    0.310485
517755    0.117102
517756    0.181581
517757    0.318203
517758    0.408324
            ...   
690334    0.101936
690335    0.501843
690336    0.243088
690337    0.128560
690338    0.491192
Name: accident_risk, Length: 172585, dtype: float32

In [14]:
!rm submission.csv
sub_df['Price'] = autogluon_preds.values
sub_df.to_csv('submission.csv', index=False)
sub_df.head()

rm: cannot remove 'submission.csv': No such file or directory


,id,accident_risk,Price
0,517754,0.352,0.310485
1,517755,0.352,0.117102
2,517756,0.352,0.181581
3,517757,0.352,0.318203
4,517758,0.352,0.408324
